In [82]:
import pandas as pd
import numpy as np
import os
import re

## this function performs 'natural' or 'human' sorting
## when all files named same except for an integer
## sort_nicely() function was borrowed from StackOverflow
def sort_nicely( l ):
    """ Sort the given list in the way that humans expect.
    """
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    l.sort( key=alphanum_key )

results_dir = '//dnrsrv01/FDrive/IWMShare/Projects/Cycle_Well/LPMT/Conventional/Results/'
batch_dirs = ['Batch1/Processed','Batch2/Processed','Batch3/Processed','Batch4/Processed','Batch5/Processed','Batch6/Processed',
             'CompletedOutsideInterest/Processed','FinishedFinalCells/Processed']
os.chdir(results_dir)

In [51]:
# files = os.listdir(os.path.join(results_dir,batch_dirs[path]))
# ## filter files
# rng = range(5,60)
# rng_string = [str(num) for num in rng]
# files = [file for file in files if any(num in file for num in rng_string)]
# sort_nicely(files)
# files
# pd.read_csv(files[0])

## This script pulls the sum column for all rows of a spreadsheet
### As written it will search through all CSV's with integers 5-59 in the filename

In [85]:
##Creates new spreadsheets to be appended together that display the SDF
##values for each r_c
##Must be checked for duplicates.
batches = ['batch1','batch2','batch3','batch4','batch5','batch6','batch7','batch8']
for path in range(0,len(batch_dirs)):
    current = os.path.join(results_dir,batch_dirs[path])
    os.chdir(current)
    files = os.listdir(current)
    output = batches[path]
    ## filter files
    rng = range(5,60)
    rng_string = [str(num) for num in rng]
    files = [file for file in files if any(num in file for num in rng_string)]
    sort_nicely(files)
    flag = 0
    df_out = pd.DataFrame()
    ## get last column value for all row cols in files list
    for i in range(0,len(files)):
        with open(files[i],'r') as f:
            df = pd.read_csv(f,index_col=0)
            if flag == 0:
                df_out['R_C'] = df.index
                df_out.set_index('R_C',inplace=True)
                flag = 1
            df_out[str(i+5)] = df['599']
    
    df_final = pd.DataFrame()
    df_final['R_C'] = df_out.index
    df_final.set_index('R_C',inplace=True)
    df_final.sort_index(inplace=True)
    df_final['SumOfZones'] = df_out.sum(axis=1)
    df_final.to_csv('//dnrsrv01/FDrive/IWMShare/Projects/Cycle_Well/LPMT/Conventional/'+output+'sort.csv')
    
    
    print('Batch'+str(path+1)+'/8')

Batch1/8
Batch2/8
Batch3/8
Batch4/8
Batch5/8
Batch6/8
Batch7/8
Batch8/8


In [86]:
## Create list of file names
uniq = []
for i in range(1,60):
    a = 'SDFZone'
    b = str(i)+'.csv'
    c = a+b
    uniq.append(c)

In [88]:
## Combine Zone files from separate batches
## could possible check for duplicates here too but 
## easier to just do in the next Cell.
alldfs = []

for filename in uniq:
    alldfs = []
    for path in range(0,len(batch_dirs)):
        current = os.path.join(results_dir,batch_dirs[path])
        os.chdir(current)
        df = pd.read_csv(filename, index_col=None, header=0)
        alldfs.append(df)

    frame = pd.concat(alldfs, axis=0, ignore_index=True)
    os.chdir('//dnrsrv01/FDrive/IWMShare/Projects/Cycle_Well/LPMT/Conventional/Results/byZone')
    frame.to_csv(filename)
    

In [89]:
## After combining files above, this loop remove duplicates
## items to change based on input: dropdup.drop('this_string',axis=1, inplace=True)
##                                 dropdup.rename({'this_string':'this_string'},axis='columns',inplace=True) # or remove this line
##                                 uniq = ['list','of','filenames']
##                                 os.chdir('this/absolute/or/relative/path')
## double check file size after - previous issue with file of all zeroes (maybe np.zeroes would fix?)
## Last run 3/13/2019 it handled the zeroes folder no problem
for filename in uniq:
    os.chdir('//dnrsrv01/FDrive/IWMShare/Projects/Cycle_Well/LPMT/Conventional/Results/byZone')
    dropdup = pd.read_csv(filename)
    dropdup.drop('Unnamed: 0',axis=1,inplace=True)
    dropdup.rename({'Unnamed: 0.1':'R_C'},axis='columns',inplace=True)
    dropdup.drop_duplicates(keep='first',inplace=True)
    os.chdir('//dnrsrv01/FDrive/IWMShare/Projects/Cycle_Well/LPMT/Conventional/Results/byZone/removeDuplicates')
    dropdup.to_csv(filename)